In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./ggwp_team/data/DatasetV4.csv')
df['client_id'] = df.index
df.head()

,Client ID,Gender,Year,Trip Type,Order Type,Country,Region,Star Rate,Group Type,Meal Option,...,N Tourists,Adult Tourists,Baby Tourists,Kid Tourists,Birthday,Order,Order Date,Trip Start Date,Price,Booking Depth
0,4197429,1,3,0,0,0,0,2,0,1,...,2,0,0,2,672537600.0,4803671,1.612731e+09,1.620086e+09,1718.64,85.136111
1,4197517,1,3,0,0,0,5,1,0,2,...,1,0,0,1,342230400.0,4803915,1.612775e+09,1.614816e+09,948.45,23.622917
2,4197719,1,3,0,0,0,1,1,0,2,...,1,0,0,1,-39052800.0,4806034,1.612814e+09,1.633738e+09,2128.15,242.176389
3,4199375,1,3,0,0,0,0,1,0,2,...,0,2,0,1,-300758400.0,4807039,1.612875e+09,1.614384e+09,993.85,17.468056
4,4208819,1,3,0,0,0,3,3,2,5,...,2,3,0,0,484704000.0,4808190,1.612892e+09,1.622938e+09,1121.02,116.265278


In [68]:
from spotlight.interactions import Interactions
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import rmse_score
from spotlight.evaluation import mrr_score

models_dict = {}

In [20]:
data = Interactions(user_ids=df["client_id"].values, item_ids=df["Region"].values)
train, test = random_train_test_split(data)

In [23]:
model = ImplicitFactorizationModel(n_iter=2)
model.fit(train, verbose=True)

Epoch 0: loss 0.5512301174141246
Epoch 1: loss 0.21715091806661918


In [41]:
res = model.predict(user_ids=162130)

In [47]:
mrr = mrr_score(model, test)
mrr

In [70]:
N_ITER = 1
stop_loop_debug = 0

for column_name in df.columns[:-1]:
    stop_loop_debug += 1
    print("Training on {} feature".format(column_name))
    model = ImplicitFactorizationModel(n_iter=N_ITER)
    data = Interactions(user_ids=df["client_id"].values, item_ids=df[column_name].values)
    train, test = random_train_test_split(data)
    model.fit(train, verbose=True)
    models_dict[column_name] = {"model" : model, "train": train, "test": test}
    if stop_loop_debug == 3:
        break

Training on Year feature
Epoch 0: loss 0.7045349026690338
Training on Trip Type feature
Epoch 0: loss 0.45107137489718563
Training on Order Type feature
Epoch 0: loss 0.4367696030958164


In [73]:
model = models_dict["Order Type"]["model"]
train = models_dict["Order Type"]["train"]
test = models_dict["Order Type"]["test"]

In [74]:
mrr = mrr_score(model, test)
mrr

array([1., 1., 1., ..., 1., 1., 1.])

3881